# Текущий уровень потребительской лояльности (Net Promoter Score)

Заказчик настоящего исследования — телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. 

Перед компанией стоит задача определить текущий уровень потребительской лояльности, среди клиентов из России.
Чтобы определить уровень лояльности, клиентам задавали вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Данные по проведенному опрсу выгружены в SQLite.

Респонденты разделены на три группы, в зависимости от поставленной оценки:
9-10 баллов — «cторонники» (promoters);
7-8 баллов — «нейтралы» (passives);
0-6 баллов — «критики» (detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису).

In [15]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [16]:
path_to_db = 'telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

Подготовка данных для дашборда. 

Данные собраны и преобразованы в единую витрину на языке SQL-запросов.

In [17]:
query = """
SELECT * FROM user;
"""

In [18]:
user = pd.read_sql(query, engine)
user.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10


In [19]:
user.gender_segment.unique()

array([ 1.,  0., nan])

In [20]:
user.gender_segment.isna().sum()

1301

In [21]:
query = """
SELECT * FROM location;
"""

In [22]:
location = pd.read_sql(query, engine)
location.head(3)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия


In [23]:
query = """
SELECT * FROM age_segment;
"""

In [24]:
age_segment = pd.read_sql(query, engine)
age_segment.head(3)

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34


In [25]:
query = """
SELECT * FROM traffic_segment;
"""

In [26]:
traffic_segment = pd.read_sql(query, engine)
traffic_segment.head(3)

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1


In [27]:
query = """
SELECT * FROM lifetime_segment;
"""

In [28]:
lifetime_segment = pd.read_sql(query, engine)
lifetime_segment.head(3)

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3


In [29]:
query = """
SELECT u.user_id, 
       u.lt_day, 
       CASE 
       WHEN u.lt_day <= 365 THEN 'new'
       ELSE 'used'
       END AS is_new,
       u.age,
       CASE
       WHEN u.gender_segment = 1 THEN 'female'
       WHEN u.gender_segment = 1 THEN 'man'
       ELSE 'unknown'
       END AS gender_segment,
       u.os_name, 
       u.cpe_type_name,
       l.country, 
       l.city,
       ags.title AS age_segment,
       tr.title AS traffic_segment,
       lt.title AS lifetime_segment,
       u.nps_score,
       CASE
       WHEN u.nps_score >= 9 THEN 'promoters'
       WHEN u.nps_score >= 7 THEN 'passives'
       ELSE 'detractors'
       END AS nps_group
FROM user AS u
JOIN location AS l ON u.location_id = l.location_id
JOIN age_segment AS ags ON u.age_gr_id = ags.age_gr_id
JOIN traffic_segment AS tr ON u.tr_gr_id = tr.tr_gr_id
JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id;
"""

In [30]:
telecomm_csi_tableau = pd.read_sql(query, engine)
telecomm_csi_tableau.sample(20)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
93305,ET242P,1045,used,40.0,female,ANDROID,SMARTPHONE,Россия,Новосибирск,04 35-44,05 5-10,07 25-36,10,promoters
420999,VS2EVT,2818,used,30.0,unknown,ANDROID,SMARTPHONE,Россия,Липецк,03 25-34,04 1-5,08 36+,9,promoters
60270,D3QW26,5380,used,59.0,female,ANDROID,SMARTPHONE,Россия,Волгоград,06 55-64,06 10-15,08 36+,10,promoters
175459,J23AH3,532,used,41.0,unknown,ANDROID,SMARTPHONE,Россия,НижнийНовгород,04 35-44,10 30-35,06 13-24,7,passives
495697,ZN03RZ,253,new,61.0,female,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,04 1-5,05 7-12,5,detractors
474003,YIVF2U,2884,used,51.0,unknown,ANDROID,SMARTPHONE,Россия,Хабаровск,05 45-54,04 1-5,08 36+,5,detractors
234860,M56TQF,5139,used,56.0,female,ANDROID,SMARTPHONE,Россия,Екатеринбург,06 55-64,04 1-5,08 36+,1,detractors
471069,YDJYM6,1709,used,47.0,unknown,ANDROID,SMARTPHONE,Россия,Москва,05 45-54,04 1-5,08 36+,10,promoters
27479,BF7DLB,1377,used,33.0,unknown,ANDROID,SMARTPHONE,Россия,Калининград,03 25-34,03 0.1-1,08 36+,4,detractors
134478,GXLQJQ,1484,used,30.0,female,ANDROID,SMARTPHONE,Россия,Пермь,03 25-34,08 20-25,08 36+,10,promoters


Итоговые данные использованы для построения дашборда.

In [31]:
telecomm_csi_tableau.to_csv('telecomm_csi_tableau.csv', index=False)

Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса.

Дашборд

<https://public.tableau.com/app/profile/valentin.morozov/viz/NPS_16355241193830/NPSstory?publish=yes>

Презентация

<https://yadi.sk/d/qTYV6dvMTPOc6w>